# Testing Aigents Cognitive Distortions Analysis API

Resources:
- Basic paper: https://www.pnas.org/doi/10.1073/pnas.2102061118
- Descriptions: https://t.me/collectivei/5401
- All emojis: https://emojidb.org/language-emojis
- Related: https://github.com/YisroelLen/Cognitive-Distortions-LFS


## Rendering distortions
- 😱🌋🌊🌪☄️📛Catastrophizing: Exaggerating the importance of negative events 
- ⚖️⚪️⚫️☯️Dichotomous Reasoning: Thinking that an inherently continuous situation can only fall into two categories 
- 👎💩Disqualifying the Positive: Unreasonably discounting positive experiences
- 💔💖💓🖤Emotional Reasoning: Thinking that something is true based on how one feels, ignoring the evidence to the contrary
- 🔮Fortune-telling: Making predictions, usually negative ones, about the future 
- 🏷Labeling and mislabeling: Labeling yourself or others while discounting evidence that could lead to less disastrous conclusions
- 🔬🔭🔎Magnification and Minimization: Magnifying negative aspects or minimizing positive aspects
- 🙅Mental Filtering: Paying too much attention to negative details instead of the whole picture
- 😎🧙‍♂️🧘‍♂️👳🏽Mindreading: Believing you know what others are thinking
- 📉Overgeneralizing: Making sweeping negative conclusions based on a few examples
- 🙇🏼🤦🏼Personalizing: Believing others are behaving negatively because of oneself, without considering more plausible or external explanations for behavior
- 📌🗜🚫Should statements: Having a fixed idea on how you and/or others should behave
- 🎭😊😟🤼‍♂️🤼‍♀️ Contradictive statements: Opposition of positive and negative references to the same situation or context
- 😊Positive statements: Emotionally positive sentiment about situation or context 
- 😟Negative statements: Emotionally negtative sentiment about situation or context 
- 😳Rude statements: Emotionally rude (obscene vocabulary) expressions about situation or context
    
Source: https://emojidb.org/

## Distortions in Russian (Google Translated)

- 😱🌋🌊🌪☄️📛Катастрофизация: преувеличение важности негативных событий.
- ⚖️⚪️⚫️☯️Дихотомическое мышление: мысль о том, что непрерывная по своей сути ситуация может делиться только на две категории.
- 👎💩Дисквалификация позитива: необоснованное обесценивание позитивного опыта.
- 💔💖💓🖤Эмоциональное мышление: думать, что что-то верно, основываясь на своих чувствах, игнорируя доказательства обратного.
- 🔮Гадание: предсказания будущего, обычно негативные.
- 🏷Навешивание ярлыков и неправильные ярлыки: навешивание ярлыков на себя и других, при этом игнорируя доказательства, которые могли бы привести к менее катастрофическим выводам.
- 🔬🔭🔎Увеличение и минимизация: усиление отрицательных аспектов или минимизация положительных аспектов.
- 🙅Ментальная фильтрация: слишком много внимания уделяется негативным деталям, а не всей картине.
- 😎🧙‍♂️🧘‍♂️👳🏽Чтение мыслей: вера в то, что вы знаете, о чем думают другие.
- 📉Чрезмерное обобщение: делать радикальные отрицательные выводы на основе нескольких примеров.
- 🤦🙇🏼Персонализация: убеждение, что другие ведут себя негативно из-за себя, без рассмотрения более правдоподобных или внешних объяснений поведения.
- 📌🗜🚫Утверждения о том, что следует: наличие фиксированного представления о том, как вам и/или другим следует вести себя.
- 🎭Противоречивые утверждения: Противопоставление положительных и отрицательных упоминаний об одной и той же ситуации или контексте.
- 😊Позитивные утверждения: эмоционально позитивное отношение к ситуации или контексту.
- 😟Негативные утверждения: эмоционально негативное мнение о ситуации или контексте.
- 😳Грубые высказывания: эмоционально грубые (нецензурная лексика) выражения о ситуации или контексте.


In [1]:
import os, sys, math
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

import datetime as dt

if 'pygents.aigents_api' in sys.modules:
    del sys.modules['pygents.aigents_api']
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']

from pygents.aigents_api import AigentsSentiment, PygentsSentiment, TextMetrics
from pygents.util import vector_proximity


def language_metrics(lang,metrics_list):
    metrics = {}
    for m in metrics_list:
        metrics[m] = './data/dict/' + lang + '/' + m + '.txt'
    return metrics

        
p = PygentsSentiment('./data/dict/en/positive.txt',
                     './data/dict/en/negative.txt',debug=True)
a = TextMetrics(language_metrics('en',['catastrophizing','dichotomous-reasoning']),debug=True)
s = TextMetrics(language_metrics('en',['positive','negative']),debug=True)

print(a.gram_arity)
print(s.gram_arity)


4
6


In [2]:
a.metrics


{'catastrophizing': {('will', 'be', 'a', 'catastrophe'),
  ('will', 'be', 'a', 'disaster'),
  ('will', 'be', 'horrible'),
  ('will', 'be', 'impossible'),
  ('will', 'be', 'terrible'),
  ('will', 'end'),
  ('will', 'fail'),
  ('will', 'go', 'wrong'),
  ('will', 'never', 'end'),
  ('will', 'not', 'end'),
  ('will', 'not', 'happen')},
 'dichotomous-reasoning': {('all',),
  ('always',),
  ('black', 'or', 'white'),
  ('either', 'or'),
  ('ever',),
  ('every',),
  ('everybody',),
  ('everyone',),
  ('everything',),
  ('everywhere',),
  ('neither',),
  ('never',),
  ('no', 'one'),
  ('nobody',),
  ('nor',),
  ('not', 'a', 'single'),
  ('nothing',),
  ('nowhere',),
  ('only',),
  ('perfect',),
  ('the', 'best'),
  ('the', 'worst'),
  ('worthless',)}}

In [3]:
for m in s.metrics:
    ngrams = s.metrics[m]
    for n in ngrams:
        if len(n) > 4:
            print(n)


('revelation', 'of', 'saint', 'john', 'the', 'divine')
('feel', 'like', 'a', 'million', 'dollars')
('pull', 'a', 'fast', 'one', 'on')
('respiratory', 'distress', 'syndrome', 'of', 'the', 'newborn')
('too', 'big', 'for', "one's", 'breeches')


In [4]:
texts = ['good news','bad news','good bad news','good and pleasant news about bad things','good bad cloud','empty spaces']
for t in texts:
    sentiment = p.get_sentiment(t)
    metrics = s.get_sentiment_words(t)
    assert sentiment[1] == (metrics['positive'] if 'positive' in metrics else 0) 
    assert sentiment[2] == (-metrics['negative'] if 'negative' in metrics else 0) 
    print(sentiment,metrics,t)


(0.85, 0.85, -0.0, 0.0, 2, 1) {'positive': 0.85} good news
(-0.85, 0.0, -0.85, 0.0, 2, 1) {'negative': 0.85} bad news
(0.0, 0.77, -0.77, 0.77, 3, 1) {'positive': 0.77, 'negative': 0.77, 'contradictive': 0.77} good bad news
(0.14, 0.74, -0.59, 0.66, 7, 1) {'positive': 0.74, 'negative': 0.59, 'contradictive': 0.66} good and pleasant news about bad things
(0.0, 0.77, -0.77, 0.77, 3, 1) {'positive': 0.77, 'negative': 0.77, 'contradictive': 0.77} good bad cloud
(0.0, 0.0, -0.0, 0.0, 2, 1) {} empty spaces


In [5]:
texts = ['empty spaces',
         'i will fail',
         'i will fail without of glass of water',
         'i will fail without of glass of water and bottle of whiskey',
         'i will fail without of glass of water and bottle of whiskey every day',
         'never ever',
         'damn cold']
for t in texts:
    metrics = a.get_sentiment_words(t)
    print(metrics,t)


{} empty spaces
{'catastrophizing': 0.92} i will fail
{'catastrophizing': 0.71} i will fail without of glass of water
{'catastrophizing': 0.62} i will fail without of glass of water and bottle of whiskey
{'catastrophizing': 0.59, 'dichotomous-reasoning': 0.46} i will fail without of glass of water and bottle of whiskey every day
{'dichotomous-reasoning': 1.0} never ever
{} damn cold


In [6]:
labels = ['positive','negative','rude',
'catastrophizing','dichotomous-reasoning','disqualifying-positive','emotional-reasoning','fortune-telling',
'labeling','magnification','mental-filtering','mindreading','overgeneralizing','personalizing','should-statement']
l = TextMetrics(language_metrics('en',labels),debug=False)
for t in texts:
    metrics = l.get_sentiment_words(t)
    print(metrics,t)


{} empty spaces
{'catastrophizing': 0.92} i will fail
{'catastrophizing': 0.71} i will fail without of glass of water
{'catastrophizing': 0.62} i will fail without of glass of water and bottle of whiskey
{'catastrophizing': 0.59, 'dichotomous-reasoning': 0.46} i will fail without of glass of water and bottle of whiskey every day
{'negative': 0.85, 'dichotomous-reasoning': 1.0} never ever
{'negative': 1.0, 'rude': 0.85} damn cold


In [7]:
text = ''
with open('./data/corpora/English/fishing/sample1.txt') as f:
    lines = f.readlines()
for line in lines:
    text += line
lists = {}
metrics = l.get_sentiment_words(text,lists,debug=False)
print(lists)
metrics

{'positive': [('good', 'for', 'you'), ('come', 'to'), ('thank', 'you'), ('due',), ('respect',), ('know',), ('honour',), ('highly',), ('reputable',), ('master',), ('truth',), ('just',), ('security',), ('contain',), ('know',), ('just',), ('know',), ('sincerely',), ('humble',), ('highly',), ('trust',), ('account',), ('please',), ('consider',), ('possible',), ('best',)], 'negative': [('does', 'not'), ('get', 'back'), ('have',), ('not',), ('against',), ('sad',), ('mysteriously',), ('death',), ('have',), ('died',), ('death',), ('have',), ("don't",), ('have',), ('suffered',), ('incessant',), ('crisis',), ('death',), ('sorrow',)], 'personalizing': [('because', 'i')], 'dichotomous-reasoning': [('nor',), ('only',)], 'rude': [('god',)], 'overgeneralizing': [('completely',)], 'magnification': [('best',)]}


{'positive': 0.45,
 'negative': 0.38,
 'personalizing': 0.08,
 'dichotomous-reasoning': 0.08,
 'rude': 0.05,
 'overgeneralizing': 0.05,
 'magnification': 0.05,
 'contradictive': 0.41}

In [16]:
import binascii
def byte_length(i):
    return (i.bit_length() + 7) // 8
b = binascii.crc32('positive'.encode('utf8'))
print(b,byte_length(b),hex(b))

1258198117 4 0x4afe9465


In [8]:
#TODO plots - generate color from text
#TODO icons
#TODO emotion (overall)


